In [20]:
import os
import pickle

In [27]:
def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

def log_parser(log_file):
    for line in log_file:
        if "val/clvqa/textvqa_accuracy" in line:
            return float(line[line.rindex(' ') + 1:])

def save_res(inp_list, save_loc):
    with open('./{}.pkl'.format(save_loc), 'wb') as f:
        pickle.dump(inp_list, f)

In [33]:
strategy = 'ft' # ft, ewc
num_tasks = 5

In [29]:
results_subdirectories = [
    dir for dir in get_immediate_subdirectories('results') if 'setting' in dir]
permutations = [dir[-num_tasks:] for dir in results_subdirectories]


In [30]:
t1_accuracy_dict, avg_accuracy_dict = dict(), dict()
for permutation, subdir in zip(permutations, results_subdirectories):
    res_path = 'results/{}/val_{}'.format(subdir, strategy)
    t1_accuracy_list, avg_accuracy_list = [], []
    classes_seen_so_far, task_1 = [], permutation[0]
    for task in permutation:
        classes_seen_so_far.append(task)
        sum_acc = 0.0
        for _class in classes_seen_so_far:
            res_tuple = '{}2{}'.format(task, _class)
            log_path = '{}/{}'.format(res_path, res_tuple)
            log_file = open('{}/train.log'.format(log_path))
            acc = log_parser(log_file)
            if _class == task_1:
                t1_accuracy_list.append(acc)
            sum_acc += acc
            log_file.close()
        avg_acc = sum_acc / len(classes_seen_so_far)
        avg_accuracy_list.append(avg_acc)
    t1_accuracy_dict[permutation] = t1_accuracy_list
    avg_accuracy_dict[permutation] = avg_accuracy_list
    

In [32]:
save_res(t1_accuracy_dict, "{}-t1_accuracy_dict_{}".format(strategy, 'vqa'))
save_res(avg_accuracy_dict, "{}-avg_accuracy_dict_{}".format(strategy, 'vqa'))
